The FinBERT pretrained model can be fine-tuned on downstream financial NLP tasks. This notebook illustrates the process of fine-tuning FinBERT using Huggingface 🤗's tranformers library. You can modify this notebook accordingly to meet you needs.

In [11]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [43]:
# tested in transformers==4.18.0, pytorch==1.7.1 
import torch
import transformers
torch.__version__, transformers.__version__

('1.7.1', '4.18.0')

*Note: the following code is for demonstration purpose. Please use GPU for fast inference on large scale dataset.*

In [41]:
torch.cuda.is_available()

True

### load dataset

In [51]:
df = pd.read_csv('analysttone.csv') ## use your own customized dataset
df.head()

,sentence,label
0,Detrimental margin for the Consumer and Profes...,0
1,Technology and Content Spending - Deceleration...,0
2,ASH did contribute $5 million to its asbestos ...,0
3,I mean it is the question of whether you are c...,0
4,"Since this rally, PEP's P/E multiple has contr...",0


In [52]:
df = df.dropna(subset=['sentence', 'label']) ## drop missing values

### prepare training/validation/testing

In [53]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(8096, 2) (1000, 2) (900, 2)


### load FinBERT pretrained model
The pretrained FinBERT model path on Huggingface is https://huggingface.co/yiyanghkust/finbert-pretrain

In [54]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

loading configuration file https://huggingface.co/yiyanghkust/finbert-pretrain/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/52a95139f4c3367976e7045d2f955bf82242a485e8506539b49838ca3c825654.ddd3650c75e2551777ba1291415522ee7288b429a28ea24681ddf54025802027
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,

### prepare dataset for fine-tuning

In [55]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


### define training options

In [56]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

trainer.train()   

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, __index_level_0__. If sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/root/transformers4.19/transformers4.19/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8096
  Num Ep

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.374136,0.858889
2,No log,0.330843,0.874444
3,No log,0.341537,0.864444
4,No log,0.364325,0.857778
5,No log,0.376574,0.862222


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, __index_level_0__. If sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 900
  Batch size = 32
Saving model checkpoint to temp/checkpoint-64
Configuration saved in temp/checkpoint-64/config.json
Model weights saved in temp/checkpoint-64/pytorch_model.bin
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, __index_level_0__. If sentence, __i

TrainOutput(global_step=320, training_loss=0.33510186672210696, metrics={'train_runtime': 162.1763, 'train_samples_per_second': 249.605, 'train_steps_per_second': 1.973, 'total_flos': 2662707787407360.0, 'train_loss': 0.33510186672210696, 'epoch': 5.0})

### evaluate on testing set

In [62]:
model.eval()
trainer.predict(dataset_test).metrics

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, __index_level_0__. If sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 32


{'test_loss': 0.32989031076431274,
 'test_accuracy': 0.881,
 'test_runtime': 1.4364,
 'test_samples_per_second': 696.197,
 'test_steps_per_second': 5.57}

### save the fine-tuned model

In [63]:
trainer.save_model('finbert-sentiment/')

Saving model checkpoint to finbert-sentiment/
Configuration saved in finbert-sentiment/config.json
Model weights saved in finbert-sentiment/pytorch_model.bin
